<a href="https://colab.research.google.com/github/ravidu-hevaganinge/AI_Scribe/blob/MedGPT/multi_metric_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://klu.ai/glossary/rouge-score
# See section on Alternatives to ROUGE

In [1]:
!pip install transformers datasets accelerate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/wyim/aci-bench.git

Cloning into 'aci-bench'...
remote: Enumerating objects: 528, done.
remote: Counting objects: 100% (528/528), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 528 (delta 357), reused 488 (delta 328), pack-reused 0
Receiving objects: 100% (528/528), 8.98 MiB | 11.69 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=148487291de8a9356bb502fe9a40cfb9ba37605c2fe8528974b2743668054189
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
# valid.csv - virtassist, D2N068
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric
import numpy as np
import torch

# Assuming you've already mounted Google Drive and set the path to your model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/UMD BIOE CAPSTONE PROJECT/Code/Summarization_BART/vitAI_BART_model", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/UMD BIOE CAPSTONE PROJECT/Code/Summarization_BART/vitAI_BART_model", local_files_only=True)

# Ensure the model is using GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load metrics
rouge = load_metric("rouge")
bleu = load_metric("bleu")
meteor = load_metric("meteor")

input_text = """[doctor] hi , brian . how are you ? [patient] hi , good to see you . [doctor] it's good to see you too . so , i know the nurse told you a little bit about dax . [patient] mm-hmm . [doctor] i'd like to tell dax about you , okay ? [patient] sure . [doctor] so , brian is a 58 year old male with a past medical history significant for congestive heart failure and hypertension , who presents today for follow-up of his chronic problems . so , brian , it's been a little while i've seen you . [patient] mm-hmm . [doctor] whats , what's going on ? [patient] i , i just feel out of sorts lately . i do n't know if it's the change in the seasons or if we're just doing a lot of projects around the house and , and some , some construction on our own . i'm just feeling out of it . lack of , uh , energy . i'm just so tired and fatigued , and i feel kinda ... i feel lightheaded every once in a while . [doctor] okay . all right . um , how long has that been going on for ? [patient] uh , probably since labor day , so about five weeks or so . [doctor] okay . and , have you noticed any , like , symptoms of weight gain , like , like swollen legs , or , you know , your belly feels bloated and things like that ? [patient] i feel , i feel bloated every once in a while . [doctor] okay . all right . um , and , are you taking your , your medications ? [patient] uh , yes , i am . [doctor] okay . and , how about your diet ? are you watching your diet ? [patient] uh , it's been a little bit of a struggle . we began construction on our kitchen over labor day weekend , and it was ... hard to cook or prepare meals so we ate out a lot, and not always the best food out. it , it , it kind of reeked havoc , uh , so it's been maybe off a little bit . [doctor] okay . all right . and , how about , you know , other symptoms , like , have you had a fever or chills ? [patient] no . [doctor] okay , and any problems breathing ? do you feel short of breath ? [patient] uh , just when i'm doing doing the projects . again , not even lifting anything really heavy , it's just that if i'm ex- exerting any energy , i , i kinda feel it at that point . [doctor] okay . do you have any chest pain ? [patient] slight cramps . that seems to go away after about , maybe about an hour or so after i first feel it . [doctor] okay , and how about a cough ? [patient] a , a slight cough , and again , i'm not sure if it's just the change of seasons and i'm getting a cold . [doctor] mm-hmm . okay . all right . well , you know , for the most part , how , you know , before all of this- [patient] mm-hmm . [doctor] . how were you doing with your heart failure ? i know that we've kinda talked about you being able to watch your healthy food intake and that's been kind of a struggle in the past . [patient] i , i , i've actually been pretty good about that ever since . the , the , the last year , it's been a little chaotic , but i wanted to make sure i stayed on top of that . [doctor] okay . all right . are you excited for halloween ? [patient] uh , ca n't wait . [doctor] okay . [patient] our home renovations should be complete by then [doctor] all right , yeah , right . [patient] yeah . [doctor] and , so , lastly , for your high blood pressure , how are you doing with that ? have , are , did you buy the blood pressure cuff like i asked ? [patient] yeah , i , i did , and we do mon- , i , i monitor it regularly . my wife makes sure i stay on top of that , but it's been pretty good . [doctor] okay . all right . well , i know you did the review of systems sheet when you checked in , and you were endorsing this fatigue- [patient] mm-hmm . [doctor] . and a little dizziness and we just talked a lot about a lot of other symptoms . [patient] mm-hmm . [doctor] any other symptoms i might be missing ? nausea or vomiting , diarrhea ? [patient] no . [doctor] anything like that ? [patient] no . [doctor] okay . all right . well , i just want to go ahead and do a quick physical exam . [patient] mm-hmm . [doctor] hey , dragon ? show me the vital signs . so , looking at your vital signs here in the office , everything looks good . you know , your blood pressure and your heart rate and your oxygenation all look really good . [patient] mm-hmm . [doctor] so , i'm gon na just take a listen to a few things and check some things out , and i'll let you know what i find , okay ? [patient] perfect . [doctor] okay . so , on your physical examination , you know , i do appreciate some jugular venous distention to- [patient] mm-hmm . [doctor] to about eight centimeters . on your heart exam , i do appreciate a three out of six systolic ejection murmur , which we've heard in the past . and , on your lung exam , i do appreciate some fine crackles at the bases bilaterally , and your lower extremities have , you know , 1+ pitting edema . so , what does all that mean ? that means i think you're retaining a little bit of fluid . [patient] mm-hmm . [doctor] okay ? i wan na just go ahead and look at some of your results , okay ? [patient] sure . [doctor] hey , dragon ? show me the chest x-ray . so , looking here at the results of your chest x-ray , it does look like you have a little bit of fluid in your lungs there , and that can be just from , um , your heart failure , okay ? hey , dragon ? show me the echocardiogram . so , this is the echocardiogram that we did about four months ago , and this shows that the pumping function of your heart is a little bit reduced at 45 % , and it also shows that leaky valve , the mitral regurgitation that , that you have , okay ? um , so , let me just go over and talk about , a little bit , my assessment and my plan for you . [patient] mm-hmm . [doctor] okay ? so , for your first problem , your congestive heart failure , i think you're retaining fluid , and i wan na go ahead and increase your lasix to 80 mg once a day . [patient] mm-hmm . [doctor] i want you to weigh yourself every day . i want you to call me if you're gaining more weight . [patient] mm-hmm . [doctor] and , i certainly want you to call me if you have any other symptoms of shortness of breath , and i wan na go ahead and order another echocardiogram , okay ? [patient] sure . [doctor] hey , dragon ? order an echocardiogram . lastly , for your high blood pressure , it looks like you're managing it well at this time , okay ? so , i wan na go ahead and continue with the lisinopril 20 mg a day . i want you to continue to record your blood pressures at home , and report them to me in the patient portal if you see they're getting elevated , okay ? [patient] mm-hmm . [doctor] does that sound like a plan ? [patient] that sounds fine . [doctor] okay . um , i'm gon na be in touch with you after we get your test results , and we'll go from there , okay ? [patient] sure . [doctor] all right . hey , dragon , finalize the note ."""
reference_summary = """CHIEF COMPLAINT Follow-up of chronic problems. HISTORY OF PRESENT ILLNESS Brian White is a 58-year-old male with a past medical history significant for congestive heart failure and hypertension, who presents today for follow-up of his chronic problems. The patient states he has been feeling out of sorts lately. He is not sure if it is due to the change in the seasons or due to performing lots of projects and some construction on his home. He reports fatigue and lightheadedness. This has been going on for about 5 weeks. While exerting energy, he has experienced some shortness of breath and chest cramps. The patient also notes a slight cough, but he is not sure if it is just the change in seasons. He feels bloated every once in a while. His diet has been a little bit of a struggle. They had construction on their kitchen begin over Labor Day weekend, and have been eating less healthy food as a result. Regarding his heart failure, he has been pretty good with his salt intake. He has been pretty good about his diet since the last year and is staying on top of that as much as possible. The patient has continued to utilize Lasix daily. For his hypertension, this has been well controlled with lisinopril 20 mg a day. He has continued to monitor his blood pressure regularly. The patient did the review of systems sheet when he checked in. He denies weight gain, swelling in the lower extremities, fevers, chills, dizziness, nausea, vomiting, and diarrhea."""  # Add your reference summary here

inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
inputs = inputs.to(device)

# Generation parameters for longer outputs
gen_kwargs = {
    "max_length": 256,
    "min_length": 150,
    "length_penalty": 2.0,
    "num_beams": 4,
    "early_stopping": True,
}

# Generate summary
summary_ids = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], **gen_kwargs)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated Summary:\n", generated_summary)

# Evaluate ROUGE
rouge_scores = rouge.compute(predictions=[generated_summary], references=[reference_summary])
print("ROUGE Scores:\n", rouge_scores)

# For BLEU and METEOR, we need to tokenize the summaries
tokenized_generated_summary = tokenizer.tokenize(generated_summary)
tokenized_reference_summary = [tokenizer.tokenize(reference_summary)]  # BLEU expects a list of possible references

# Evaluate BLEU
bleu_score = bleu.compute(predictions=[tokenized_generated_summary], references=[tokenized_reference_summary])
print("BLEU Score:\n", bleu_score)

# Evaluate METEOR
meteor_score = meteor.compute(predictions=[generated_summary], references=[reference_summary])
print("METEOR Score:\n", meteor_score)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major r

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Generated Summary:
 CHIEF COMPLAINT

HISTORY OF PRESENT ILLNESS

Brian is a 58-year-old male with a past medical history significant for congestive heart failure and hypertension who presents today for follow-up of his chronic problems.

SOCIAL HISTORY

The patient reports that he has been feeling out of sorts lately. He is doing a lot of projects around the house and some construction on his own. He feels tired and fatigued, and he feels kinda lightheaded every once in a while. The symptoms have been going on for about 5 weeks. He denies chest pain, slight cramps, fever, chills, cough, or shortness of breath. He has been taking his medications regularly. The patient has been able to watch his healthy food intake and has been pretty good about that ever since the last year, but he wanted to make sure he stayed on top of that. He states that it has been a little bit of a struggle since he began construction of his kitchen over Labor Day weekend, and it was hard to cook or prepare meals,